# Exercise on prompt evals

## Exercise Task

Give the Model Grader more context on what a good solution looks like 

In [1]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-haiku-4-5"

In [2]:
# Helper functions
def add_messages(messages, text, role="user"):
    message = {"role": role, "content": text}
    messages.append(message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[]):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message.content[0].text

In [3]:
# Function to generate a new dataset
import json


def generate_dataset():
    prompt = """
Generate a evaluation dataset for a prompt evaluation. The dataset will be used to evaluate prompts
that generate Python, JSON, or Regex specifically for AWS-related tasks. Generate an array of JSON objects,
each representing task that requires Python, JSON, or a Regex to complete.

Example output:
```json
[
    {
        "task": "Description of task",
        "format": "json" or "python" or "regex",
        "solution_criteria": "Key criteria for evaluating the solution"
    },
    ...additional
]
```

* Focus on tasks that can be solved by writing a single Python function, a single JSON object, or a regular expression.
* Focus on tasks that do not require writing much code

Please generate 3 objects.
"""

    messages = []
    add_messages(messages, prompt, role="user")
    add_messages(messages, "```json", role="assistant")
    text = chat(messages, stop_sequences=["```"])
    return json.loads(text)

In [4]:
# Generate the dataset and write it to 'dataset.json'
dataset = generate_dataset()
with open("dataset_exercise.json", "w") as f:
    json.dump(dataset, f, indent=4)

In [5]:
# Function to grade a test case + output using a model
def grade_by_model(test_case, output):
    eval_prompt = f"""
You are an expert AWS code reviewer. Your task is to evaluate the following AI-generated solution.

Original Task:
<task>
{test_case["task"]}
</task>

Solution to Evaluate:
<solution>
{output}
</solution>

Criteria you should use to evaluate the solution:
<criteria>
{test_case["solution_criteria"]}
</criteria>


Output Format
Provide your evaluation as a structured JSON object with the following fields, in this specific order:
- "strengths": An array of 1-3 key strengths
- "weaknesses": An array of 1-3 key areas for improvement
- "reasoning": A concise explanation of your overall assessment
- "score": A number between 1-10

Respond with JSON. Keep your response concise and direct.
Example response shape:
{{
    "strengths": string[],
    "weaknesses": string[],
    "reasoning": string,
    "score": number
}}
    """

    messages = []
    add_messages(messages, eval_prompt, role="user")
    add_messages(messages, "```json", role="assistant")
    eval_text = chat(messages, stop_sequences=["```"])
    return json.loads(eval_text)

In [6]:
# Passes a test case into Claude
def run_prompt(test_case):
    prompt = f"""
Please solve the following task:

{test_case["task"]}

* Respond only with Python, JSON, or a plain Regex
* Do not add any comments or commentary or explanation
"""

    messages = []
    add_messages(messages, prompt, role="user")
    add_messages(messages, "```code", role="assistant")
    output = chat(messages, stop_sequences=["```"])
    return output

In [7]:
# Functions to validate the output structure
import re
import ast


def validate_json(text):
    try:
        json.loads(text.strip())
        return 10
    except json.JSONDecodeError:
        return 0


def validate_python(text):
    try:
        ast.parse(text.strip())
        return 10
    except SyntaxError:
        return 0


def validate_regex(text):
    try:
        re.compile(text.strip())
        return 10
    except re.error:
        return 0


def grade_syntax(response, test_case):
    format = test_case["format"]
    if format == "json":
        return validate_json(response)
    elif format == "python":
        return validate_python(response)
    else:
        return validate_regex(response)


In [8]:
# Function to execute a single test case and grade the output
def run_test_case(test_case):
    """Calls run_prompt, then grades the result"""
    output = run_prompt(test_case)

    model_grade = grade_by_model(test_case, output)
    model_score = model_grade["score"]
    reasoning = model_grade["reasoning"]

    syntax_score = grade_syntax(output, test_case)

    score = (model_score + syntax_score) / 2

    return {
        "output": output,
        "test_case": test_case,
        "score": score,
        "reasoning": reasoning,
    }

In [9]:
from statistics import mean


def run_eval(dataset):
    """Loads the dataset and calls run_test_case with each case"""
    results = []

    for test_case in dataset:
        result = run_test_case(test_case)
        results.append(result)

    average_score = mean([result["score"] for result in results])
    print(f"Average score: {average_score}")

    return results

In [10]:
with open("dataset_exercise.json", "r") as f:
    dataset = json.load(f)

results = run_eval(dataset)

Average score: 6.666666666666667


In [11]:
print(json.dumps(results, indent=2))

[
  {
    "output": "\nimport re\n\ndef parse_s3_bucket(uri):\n    match = re.match(r's3://([a-z0-9.-]+)(/|$)', uri)\n    return match.group(1) if match else None\n",
    "test_case": {
      "task": "Parse an AWS S3 bucket name from a full S3 URI path (e.g., 's3://my-bucket-name/folder/file.txt') and extract just the bucket name",
      "format": "regex",
      "solution_criteria": "The regex should correctly extract bucket names from S3 URIs, handling buckets with hyphens and numbers, and should not match the path components after the bucket name"
    },
    "score": 8.5,
    "reasoning": "The solution effectively handles the core requirement of extracting bucket names while excluding path components. However, it has notable limitations: the lowercase-only character class is unnecessarily restrictive compared to actual S3 bucket naming rules, and there is no validation of bucket name constraints. While these issues may not affect typical well-formed URIs, they represent edge cases an